In [2]:
!pip install transformers
!pip install datasets
!pip install py7zr
!pip install sentencepiece


from datasets import load_dataset
from transformers import T5Tokenizer, T5ForConditionalGeneration

# Load the dataset
dataset = load_dataset("samsum")
train_data = dataset["train"]

# Load the tokenizer for T5-large
tokenizer = T5Tokenizer.from_pretrained("t5-base")

def preprocess_function(examples):
    inputs = examples["dialogue"]
    targets = examples["summary"]
    inputs = [doc + tokenizer.eos_token for doc in inputs]

    # Adjust max_length for T5-large
    model_inputs = tokenizer(inputs, max_length=1024, padding=True, truncation=True, return_tensors="pt")

    # Setup the target for the loss (classification)
    with tokenizer.as_target_tokenizer():
        # Adjust max_length for T5-large
        labels = tokenizer(targets, max_length=300, padding=True, truncation=True, return_tensors="pt")

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Preprocess the data
train_data = train_data.map(preprocess_function, batched=True)

# Load the T5-large model
model = T5ForConditionalGeneration.from_pretrained("t5-base")

# Rest of the code for fine-tuning and training remains the same


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 29.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 32.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 44.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 65.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 492.4/492.4 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.3/412.3 kB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.8/138.8 kB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Generating train split:   0%|          | 0/14732 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/819 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/818 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5.py:199: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(
You are using the legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This means that tokens that come after special tokens will not be properly handled. We recommend you to read the related pull request available 

Map:   0%|          | 0/14732 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5.py:262: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3635: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import torch
from torch.utils.data import DataLoader
from transformers import T5ForConditionalGeneration, T5Tokenizer, AdamW
from datasets import load_dataset

# Load the dataset
dataset = load_dataset("samsum")
train_data = dataset["train"]

# Load the pre-trained T5-base model and tokenizer
model = T5ForConditionalGeneration.from_pretrained("t5-base")
tokenizer = T5Tokenizer.from_pretrained("t5-base")

# Define the optimizer
optimizer = AdamW(model.parameters(), lr=1e-4)

# Fine-tune the model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Create a custom collate function to pad the sequences within the batch
def collate_fn(batch):
    input_text = [item["dialogue"] for item in batch]
    target_text = [item["summary"] for item in batch]

    model_inputs = tokenizer(input_text, padding=True, return_tensors="pt", truncation=True)
    labels = tokenizer(target_text, padding=True, return_tensors="pt", truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    model_inputs["labels"] = model_inputs["labels"].to(device)
    model_inputs["input_ids"] = model_inputs["input_ids"].to(device)
    model_inputs["attention_mask"] = model_inputs["attention_mask"].to(device)

    return model_inputs

# Assuming you have defined train_data and batch_size for the fine-tuning data
batch_size = 8  # Define the batch size as desired
train_dataloader = DataLoader(train_data, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)

num_epochs = 5
for epoch in range(num_epochs):
    model.train()  # Set the model to training mode for each epoch
    total_loss = 0.0
    for batch in train_dataloader:
        input_ids = batch["input_ids"]
        attention_mask = batch["attention_mask"]
        labels = batch["labels"]

        optimizer.zero_grad()
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_epoch_loss = total_loss / len(train_dataloader)
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {avg_epoch_loss}")

# Save the fine-tuned model and tokenizer to Google Drive
output_dir = "/content/drive/My Drive/NLP_Project_1/FineTunedModel_T5_Base/"
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)


/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5.py:199: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprec

Epoch 1/5, Loss: 0.9622506745863427
Epoch 2/5, Loss: 0.783759183850014
Epoch 3/5, Loss: 0.7072533038520658
Epoch 4/5, Loss: 0.6429181087127078
Epoch 5/5, Loss: 0.5860451110896536


('/content/drive/My Drive/NLP_Project_1/FineTunedModel_T5_Base/tokenizer_config.json',
 '/content/drive/My Drive/NLP_Project_1/FineTunedModel_T5_Base/special_tokens_map.json',
 '/content/drive/My Drive/NLP_Project_1/FineTunedModel_T5_Base/spiece.model',
 '/content/drive/My Drive/NLP_Project_1/FineTunedModel_T5_Base/added_tokens.json')

In [4]:
!pip install rouge-score


  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=8c77ae8b57334890c650777acd17e2af9896c924b3ca4da743479550d03f4179
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


In [8]:
from rouge_score import rouge_scorer

# Define a function to generate summaries using the fine-tuned model
def generate_summary(input_ids, attention_mask):
    summary_ids = model.generate(input_ids=input_ids, attention_mask=attention_mask)
    return tokenizer.batch_decode(summary_ids, skip_special_tokens=True)

# Calculate the ROUGE score for the entire dataset
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

references = []  # List to store the ground truth summaries
predictions = []  # List to store the predicted summaries

for batch in train_dataloader:
    input_ids = batch["input_ids"]
    attention_mask = batch["attention_mask"]
    labels = batch["labels"]

    # Convert the tensors to text using the tokenizer
    input_text = tokenizer.batch_decode(input_ids, skip_special_tokens=True)
    target_text = tokenizer.batch_decode(labels, skip_special_tokens=True)

    reference_summaries = [summary.strip() for summary in target_text]
    predicted_summaries = generate_summary(input_ids, attention_mask)

    references.extend(reference_summaries)
    predictions.extend(predicted_summaries)

# Calculate the ROUGE scores for each pair of generated and reference summaries
rouge1_scores = []
rouge2_scores = []
rougeL_scores = []

for generated, reference in zip(predictions, references):
    scores = scorer.score(generated, reference)
    rouge1_scores.append(scores["rouge1"].fmeasure)
    rouge2_scores.append(scores["rouge2"].fmeasure)
    rougeL_scores.append(scores["rougeL"].fmeasure)

# Calculate the overall ROUGE score (average of all ROUGE scores)
overall_rouge1_score = sum(rouge1_scores) / len(rouge1_scores)
overall_rouge2_score = sum(rouge2_scores) / len(rouge2_scores)
overall_rougeL_score = sum(rougeL_scores) / len(rougeL_scores)

print("ROUGE-1: {:.4f}".format(overall_rouge1_score))
print("ROUGE-2: {:.4f}".format(overall_rouge2_score))
print("ROUGE-L: {:.4f}".format(overall_rougeL_score))

# Calculate the overall ROUGE score (average of all ROUGE scores)
overall_rouge_score = (overall_rouge1_score + overall_rouge2_score + overall_rougeL_score) / 3
print("Overall ROUGE score: {:.4f}".format(overall_rouge_score))


ROUGE-1: 0.5074
ROUGE-2: 0.2872
ROUGE-L: 0.4406
Overall ROUGE score: 0.4117
